In [ ]:
#!/usr/bin/env


"""


split, clean, and self-cal continuum and line data
NOTE: this is intended to be an interactive, iterative process
      so this is more a log that should be run by cutting and
      pasting into casa rather than as an executable script
      search "CHANGEME" for variables to be changed
10/9/15 MCA
"""


#  Setup 

LupusIII_71   16:08:54.68   -39:37:43.9
Class II, M1


In [ ]:
field = 10                                     # CHANGEME 
file_ms = '../science_calibrated.ms'
contspw   = '2,3,4,7,8,9'                      # continuum spectral windows
contspw_w = [128,3840,1920,128,3840,1920]      # continuum spw widths
robust     = 0.5                               # CHANGEME
imsize     = [640,640]
cell       = '0.03arcsec'
imagermode = 'csclean'
refant     = 'DA52'                            # CHANGEME
xc         = 322                               # CHANGEME
yc         = 333                               # CHANGEME
in_a       = 80
out_a      = 120
aper       = 1.25
boxwidth = 300.
box = rg.box([xc-boxwidth,yc-boxwidth],[xc+boxwidth,yc+boxwidth])


#  Split Off Continuum 

split off field from full ms


In [ ]:
split(vis = file_ms,
      outputvis = 'f'+str(field)+'.vis',
      field = field,
      datacolumn = 'data')


split off continuum (take the large bw spw and average


In [ ]:
split(vis = 'f'+str(field)+'.vis',
      outputvis = 'f'+str(field)+'_cont.vis',
      spw = contspw,
      width = contspw_w,
      datacolumn = 'data')


plot uv-distance vs. amplitude


In [ ]:
plotms(vis='f'+str(field)+'_cont.vis',
       xaxis='uvdist',yaxis='amp',
       coloraxis='spw')
       # plotfile='f'+str(field)+'_ampuv_orig.png'
       # showgui=False,
       # highres=True,
       # overwrite=True)


source is resolved
find antenna close to center of configuration
check pipeline log that this ant is OK


In [ ]:
plotants(vis='f'+str(field)+'_cont.vis') #, figfile='f'+str(field)+'_ants.png')


#  Clean continuum before selfcal 

os.system('rm -rf f'+str(field)+'_cont_b4sc*')
light clean (100 iterations) to set the mask around the main peaks


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_b4sc',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 100,
      threshold = '0.0mJy',
      interactive = True,
      mask = '',
      cell = cell,
      imsize = imsize,
      weighting = 'briggs',
      robust = robust,
      imagermode = imagermode)
im_max = imstat(imagename = 'f'+str(field)+'_cont_b4sc.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_b4sc.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)


Peak = 33.27 mJy, rms = 0.36 mJy, S/N = 92.2


#  Self-Calibrate 1 

first combine all the data by time (solint = inf)
i.e., phase self-cal over entire integration time


In [ ]:
gaincal(vis = 'f'+str(field)+'_cont.vis',
        caltable = 'f'+str(field)+'_cont_pcal1',
        refant = refant,
        solint = 'inf',
        combine = 'spw',
        gaintype = 'T',
        spw = '',
        calmode = 'p',
        minblperant = 4,
        minsnr = 3)


plot phase for each antenna


In [ ]:
plotcal(caltable = 'f'+str(field)+'_cont_pcal1',
        xaxis = 'time',
        yaxis = 'phase',
        spw = '',
        iteration = 'antenna',
        subplot = 421,
        plotrange = [0,0,-200,200]) 


apply calibration to data


In [ ]:
applycal(vis = 'f'+str(field)+'_cont.vis',
        spw = '',
        gaintable = ['f'+str(field)+'_cont_pcal1'],
        spwmap = [0,0,0,0,0,0],
        calwt = T,
        flagbackup = F)


clean self-calibrated data


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_pcal1_clean',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 100,
      threshold   = '0.0mJy',
      interactive = False,
      mask = 'f'+str(field)+'_cont_b4sc.mask',
      cell        = cell,
      imsize      = imsize,
      weighting   = 'briggs',
      robust      = robust,
      imagermode  = imagermode)
im_max = imstat(imagename = 'f'+str(field)+'_cont_pcal1_clean.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_pcal1_clean.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)


Peak = 33.60 mJy, rms = 0.34 mJy, S/N = 99.4
inspect images


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cont_b4sc.image'},
               {'file':'f'+str(field)+'_cont_pcal1_clean.image'}])


slightly less background noise in pcal1


#  Self-Calibrate 2 

decrease phase self-cal solution interval to a few times integration time
int = 6s (from X125.log)


In [ ]:
gaincal(vis = 'f'+str(field)+'_cont.vis',
        caltable = 'f'+str(field)+'_cont_pcal2',
        refant = refant,
        solint = '20s',                           # CHANGEME
        combine = 'spw',
        gaintype = 'T',
        spw = '',
        calmode = 'p',
        minblperant = 4,
        minsnr = 3)
plotcal(caltable = 'f'+str(field)+'_cont_pcal2',
        xaxis = 'time',
        yaxis = 'phase',
        spw = '',
        iteration = 'antenna',
        subplot = 421,
        plotrange = [0,0,-200,200])
applycal(vis = 'f'+str(field)+'_cont.vis',
        spw = '',
        gaintable = ['f'+str(field)+'_cont_pcal2'],
        spwmap = [0,0,0,0,0,0],
        calwt = T,
        flagbackup = F)
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_pcal2_clean',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 100,
      threshold   = '0.0mJy',
      interactive = False,
      mask = 'f'+str(field)+'_cont_b4sc.mask',
      cell        = cell,
      imsize      = imsize,
      weighting   = 'briggs',
      robust      = robust,
      imagermode  = imagermode)
im_max = imstat(imagename = 'f'+str(field)+'_cont_pcal2_clean.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_pcal2_clean.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)


Peak = 33.82 mJy, rms = 0.34 mJy, S/N = 99.7 (same as pcal2)
inspection of the image shows no change from pcal1


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cont_b4sc.image'},
               {'file':'f'+str(field)+'_cont_pcal1_clean.image'},
               {'file':'f'+str(field)+'_cont_pcal2_clean.image'}])


pcal2 actually looks best


#  Best Continuum Map 

so now run the same applycal but with flagbackup = T,


In [ ]:
applycal(vis = 'f'+str(field)+'_cont.vis',
        spw = '',
        gaintable = ['f'+str(field)+'_cont_pcal2'],         # CHANGEME
        spwmap = [0,0,0,0,0,0],
        calwt = T,
        flagbackup = T)


deep clean, trying different robust weights


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_best',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 2000,
      threshold   = '0.0mJy',
      interactive = True,
      mask = '',
      cell        = cell,
      imsize      = imsize,
      weighting   = 'briggs',
      robust      = -1,                                      # CHANGEME
      imagermode  = imagermode)


placed mask around outer continuum contour
stopped after 700 iterations once the inside became green


In [ ]:
im_max = imstat(imagename = 'f'+str(field)+'_cont_best.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
bmaj = imhead(imagename = 'f'+str(field)+'_cont_best.image', mode="get", hdkey="beammajor")
bmin = imhead(imagename = 'f'+str(field)+'_cont_best.image', mode="get", hdkey="beamminor")
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)
print 'Beam = {0:.2f} x {1:.2f} arcsec'.format(bmaj.get('value'),bmin.get('value'))


robust = -1 
Peak = 31.87 mJy, rms = 0.31 mJy, S/N = 102.6
Beam = 0.32 x 0.26 arcsec
NOTE: BEAM DOESN'T GET BETTER WHEN USING -2
robust = -1 back to the same sort of S/N but the slightly smaller beam size provides a slightly
clearer view of the inner cavity so is worth the payoff
save this to a fits file


In [ ]:
exportfits(imagename='f'+str(field)+'_cont_best.image', fitsimage='f'+str(field)+'_cont.fits')


compare to before self-cal


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cont_b4sc.image'},
               {'file':'f'+str(field)+'_cont_best.image'}])


measure flux


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cont_best.image'}])
im_rms = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 179.19 mJy, rms = 0.30 mJy, S/N = 588.1
re-center image on source and use get_flux.py to get COG flux


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_cont_cropped.image',
             infile  = 'f'+str(field)+'_cont.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_cont_cropped.image',
           fitsimage = 'f'+str(field)+'_cont_cropped.fits')


In [ ]:
Measuring COG for M/f10_cont_cropped.fits
Assuming object center (300.0,300.0)
Background: 0.00 mJy/beam km/s
RMS in annulus 4.0-9.0 arcsec = 0.33 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10     6.08     0.09     67.1
   1     0.20    26.10     0.42     62.6
   2     0.30    65.84     0.39    167.9
   3     0.40   109.69     0.45    241.3
   4     0.50   146.77     0.55    265.7
   5     0.60   166.34     0.60    275.8
   6     0.70   174.66     0.61    286.0
   7     0.80   177.78     0.69    257.9
   8     0.90   178.95     0.75    239.8
   9     1.00   179.39     0.99    180.7
  10     1.10   179.35     1.05    170.5
  11     1.20   179.21     1.09    164.0
  12     1.30   179.23     1.35    132.8
  13     1.40   179.23     1.33    134.8
  14     1.50   179.36     1.35    133.0
F = 179.39 mJy
E = 0.99 mJy
S = 180.65
D = 2.00 arcsec


#  Measure flux with UVMODELFIT 

calculate offset from phase center in arcsec


In [ ]:
pixscale = 0.03             # must match 'cell'                 
dx = pixscale*(320.0-xc)    # offset to east (left)
dy = pixscale*(yc-320.0)    # offset to north (up)


measure flux as gaussian


In [ ]:
uvmodelfit(vis       = 'f'+str(field)+'_cont.vis',
           comptype  = 'G',
           sourcepar = [im_flux,dx,dy,0.5,1,0],
           varypar   = [T,T,T,T,T,T],
           niter     = 10)


In [ ]:
reduced chi2=1.54546
I = 0.201903 +/- 0.000803414
x = -0.103302 +/- 0.00131957 arcsec
y = 0.404801 +/- 0.00124197 arcsec
a = 0.890157 +/- 0.00378721 arcsec
r = 0.570511 +/- 0.00378844
p = 44.7996 +/- 0.31907 deg
higher than aperture method
much better chi2 than point-source method
16:08:54.673 -39:37:43.495
